In [ ]:
from casadi import *
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import animation, rc
from IPython.display import HTML

In [ ]:
def energy(x, v):
    u = -np.cos(x)
    t = 0.5 * v**2
    return u+t

In [ ]:
def timecrossings(arr, times, pos):
    crossings = []
    sign_arr = sign(arr)
    for ii in range(1,len(arr)):
        if sign_arr[ii-1] != sign_arr[ii]:
            if cos(pos[ii]) > -0.99 :
                crossings.append((times[ii-1] + times[ii])/2)
    return crossings

In [ ]:
def plot_results(arr, U, T, max_par, N):
    margin_ang = np.arcsin(max_par)
    timescale_x = np.linspace(0, T, N+1)
    timescale_u = np.linspace(0, T, N)
    arr_u = sol.value(U)
    plt.figure(figsize=[10,7])
    plt.plot(timescale_x,arr[:,0], label = '$x$')
    plt.plot(timescale_x,arr[:,1], label = "$x'$")
    plt.plot(timescale_u,arr_u[:], label = 'u')
    plt.plot(timescale_x,2+cos(arr[:,0]),':', label = '$2+cos(x)$')
    cross_points = timecrossings(arr[:,1], timescale_x, arr[:,0])
    plt.hlines([0,pi, -pi], 0, timescale_u[-1], 'k', 'dotted')
    plt.hlines([-max_par,max_par], 0, timescale_u[-1], 'r', 'dotted')
    plt.hlines([np.pi-margin_ang,-np.pi+margin_ang], 0, timescale_u[-1], 'g', 'dotted', label = "g_par = max par")
    plt.vlines(cross_points, -pi, pi, 'k', 'dotted')
    print(f'Max par: {max_par}, number of crossing points: {len(cross_points)}')
    plt.legend()

In [ ]:
def print_phase_space(X = np.array([[0,0]])):
    X = np.array(X)
    margin = 0.2
    max_x = max(np.max(X[:,0]), np.pi) + margin
    min_x = min(np.min(X[:,0]), -np.pi) - margin
    max_v = max(np.max(X[:,1]), 2) + margin
    min_v = min(np.min(X[:,1]), -2) - margin
    
    x_arr = np.linspace(min_x, max_x, 400)
    v_arr = np.linspace(min_v, max_v, 400)
    
    xx, vv = np.meshgrid(x_arr, v_arr)
    ee = energy(xx,vv)
    
    fig, ax = plt.subplots(figsize=(12, 12))
    ax.contour(xx,vv,ee, 10)
    ax.plot(X[:,0], X[:,1], 'r', marker = 'o', markersize = 5, linewidth = 0.7)
    ax.set_aspect('equal')
    plt.xlabel('Angle in rad')
    plt.ylabel('Angular speed in rad/s')
    plt.title('Phase space, lines of constant energy')

In [ ]:
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 200

In [ ]:
def create_anim(arr, U):
    arr = np.array(arr)
    U = np.array(U)
    fig, (ax, ax2) = plt.subplots(1,2, figsize = [15,8])

    ax.set_aspect('equal')
    ax.set_xlim(( -1.5, 1.5))
    ax.set_ylim(( -1.5, 1.5))

    circle2 = plt.Circle((0, 0), 1, color='b', ls = ":", fill=False)
    ax.add_artist(circle2)

    line, = ax.plot([], [], lw=2)
    point, = ax.plot([], [], marker='o', markersize=15, color="red")
    text = ax.text(0.2, 0, "", fontsize = 12)
    text_2 = ax.text(0.2, -0.15, "", fontsize = 12)
    text_3 = ax.text(0.2, -0.3, "", fontsize = 12)
    # Phase space
    
    margin = 0.2
    max_x = max(np.max(arr[:,0]), np.pi) + margin
    min_x = min(np.min(arr[:,0]), -np.pi) - margin
    max_v = max(np.max(arr[:,1]), 2) + margin
    min_v = min(np.min(arr[:,1]), -2) - margin
    
    x_arr = np.linspace(min_x, max_x, 400)
    v_arr = np.linspace(min_v, max_v, 400)
    
    xx, vv = np.meshgrid(x_arr, v_arr)
    ee = energy(xx,vv)
    
    ax2.contour(xx,vv,ee, 10)
    line2, = ax2.plot([], [], 'r', marker = 'o', markersize = 5, linewidth = 0.7)
    ax2.set_aspect('equal')
    ax2.set_xlabel('Angle in rad')
    ax2.set_ylabel('Angular speed in rad/s')
    ax2.set_title('Phase space, lines of constant energy')
    
    trayectory = np.zeros_like(arr)
    trayectory[:,0] = arr[0,0]
    trayectory[:,1] = arr[0,1]
    
    def init():
        line.set_data([], [])
        point.set_data([], [])
        text.set_text('')
        return (line,)
    def animate(i):
        x = [0, np.sin(arr[i,0])]
        y = [0, -np.cos(arr[i,0])]
        line.set_data(x, y)    
        point.set_data(x[1], y[1])
        text.set_text("U = %.6f" % U[i])
        text_2.set_text(r"$\dot{\theta}$" + " = %.6f" % arr[i,1])
        text_3.set_text("par g = %.6f" % x[1])
        
        trayectory[i:,0] = arr[i,0]
        trayectory[i:,1] = arr[i,1]
        line2.set_data(trayectory[:,0], trayectory[:,1])
        return (line, line2,)
    
    anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=arr.shape[0]-1, interval=20, 
                               blit=True)
    return anim

## Pendulum exercise
$$
\begin{split}\begin{array}{lc}
\begin{array}{l}
\text{minimize:} \\
x(\cdot) \in \mathbb{R}^2, \, u(\cdot) \in \mathbb{R}
\end{array}
\quad \displaystyle \int_{t=0}^{T}{cos(x_0) \, dt}
\\
\\
\text{subject to:} \\
\\
\begin{array}{ll}
\left\{
\begin{array}{l}
\dot{x}_0 = x_1 \\
\dot{x}_1 = u - sin(x_0) \\
-u_{max} \le u \le u_{max} , \quad
\end{array} \right. & \text{for} \, 0 \le t \le T \\
x_0(0)=0, \quad x_1(0)=0, x_0(T) = pi/2 , x_1(T) = 0
\end{array}
\end{array}\end{split}
$$
with $T=10$.

siendo $$x_0 = \theta$$ $$x_1 = \theta'$$

In [ ]:
x = MX.sym('x', 2)
t = MX.sym('t')
dt = MX.sym('dt')
u = MX.sym('u')

In [ ]:
rhs = vertcat(x[1], u-sin(x[0]))
#rhs = vertcat(x[1], u)
F = Function('F', [x, u], [rhs])

In [ ]:
k1 = F(x, u);
k2 = F(x + dt/2 * k1, u)
k3 = F(x + dt/2 * k2, u)
k4 = F(x + dt * k3, u);
new_x_expr = x+dt/6*(k1 +2*k2 +2*k3 +k4)

In [ ]:
new_x_expr = x + dt * F(x, u)
new_x = Function('New_x', [x, u, dt], [new_x_expr])

## Opti problem

In [ ]:
N = 200

In [ ]:
opti = Opti()
opti.solver('ipopt')

In [ ]:
X = opti.variable(N+1,2)
U = opti.variable(N)
#T = opti.variable()
T = opti.parameter()
u_m = opti.parameter()
#t_m = opti.parameter()

In [ ]:
cost = sum1(2+cos(X[:,0]))*T #**2
#cost = -sum1(X[:,0])
opti.minimize(cost)

In [ ]:
opti.subject_to(X[0,:].T == [0, 0])
opti.subject_to(cos(X[-1,0]) < -0.9999)
#opti.subject_to(T < t_m)
opti.subject_to(opti.bounded(-0.001,X[-1,1],0.001))

In [ ]:
for ii in range(N):
    opti.subject_to(X[ii+1,:].T == new_x(X[ii,:], U[ii], T/N))
    opti.subject_to(opti.bounded(-u_m,U[ii],u_m))

In [ ]:
opti.set_initial(X[:,0], np.linspace(0, pi, N+1))
opti.set_initial(X[:,1], pi/N)
#opti.set_initial(T, 50)
opti.set_value(T, 40)
max_par = 0.09
opti.set_value(u_m, max_par)

In [ ]:
sol = opti.solve()

In [ ]:
plot_results(sol.value(X), sol.value(U), sol.value(T), max_par, N)

In [ ]:
#anim = create_anim(sol)

In [ ]:
#HTML(anim.to_jshtml())

In [ ]:
results = []
for ii in [0.11, 0.105, 0.1, 0.095, 0.09, 0.085]:
    opti.set_value(u_m, ii)
    est_t = (2 + sqrt(1/ii))*8
    #opti.set_initial(T, est_t)
    opti.set_value(T, 40)
    try:
        sol = opti.solve()
    except:
        pass
    else:
        results.append([sol.value(X), sol.value(U), sol.value(T), ii, N])

In [ ]:
for res in results:
    plot_results(*res)

## Integración de la acción para comprobación

In [ ]:
def euler_step(x, u, dt):
    return x + dt * F(x, u)

In [ ]:
def rk4_step(x, u, dt):
    k1 = F(x, u);
    k2 = F(x + dt/2 * k1, u)
    k3 = F(x + dt/2 * k2, u)
    k4 = F(x + dt * k3, u);
    return x+dt/6*(k1 +2*k2 +2*k3 +k4)

In [ ]:
def integrate_euler(x_0, u, dt):
    x = [x_0,]
    for ii in range(len(u)):
        x_i = euler_step(x[-1], u[ii], dt)
        x.append(x_i)
    return horzcat(*x).T

In [ ]:
def integrate_rk4(x_0, u, dt):
    x = [x_0,]
    for ii in range(len(u)):
        x_i = rk4_step(x[-1], u[ii], dt)
        x.append(x_i)
    return horzcat(*x).T

In [ ]:
def plot_results_sim(U, arr, T, max_par, N):
    arr = np.array(arr)
    timescale_x = np.linspace(0, T, N+1)
    timescale_u = np.linspace(0, T, N)
    arr_u = np.array(U)
    plt.figure(figsize=[10,7])
    plt.plot(timescale_x,arr[:,0], label = '$x$')
    plt.plot(timescale_x,arr[:,1], label = "$x'$")
    plt.plot(timescale_u,arr_u[:], label = 'u')
    plt.plot(timescale_x,2+cos(arr[:,0]),':', label = '$2+cos(x)$')
    cross_points = timecrossings(arr[:,1], timescale_x, arr[:,0])
    plt.hlines([0,pi, -pi], 0, timescale_u[-1], 'k', 'dotted')
    plt.hlines([-max_par,max_par], 0, timescale_u[-1], 'r', 'dotted')
    plt.vlines(cross_points, -pi, pi, 'k', 'dotted')
    print(f'Max par: {max_par}, number of crossing points: {len(cross_points)}')
    plt.legend()

## Comparación entre integrar con Euler y Runge Kutta

In [ ]:
x_0 = DM([1,0])
u_0 = np.zeros(200)
xx = integrate_euler(x_0, u_0, 0.1)
print_phase_space(xx)

In [ ]:
xx = integrate_rk4(x_0, u_0, 0.1)
print_phase_space(xx)

### Usando Euler:

In [ ]:
xx = integrate_euler(DM([0,0]), sol.value(U), sol.value(T)/N)
plot_results_sim(sol.value(U), xx, sol.value(T), sol.value(max_par), N)

### Usando Runge Kutta

In [ ]:
xx = integrate_rk4(DM([0,0]), sol.value(U), sol.value(T)/N)
plot_results_sim(sol.value(U), xx, sol.value(T), sol.value(max_par), N)

## Otras comprobaciones: Coste = T

In [ ]:
N = 300

k1 = F(x, u);
k2 = F(x + dt/2 * k1, u)
k3 = F(x + dt/2 * k2, u)
k4 = F(x + dt * k3, u);
new_x_expr = x+dt/6*(k1 +2*k2 +2*k3 +k4)

new_x_expr = x + dt * F(x, u)
new_x = Function('New_x', [x, u, dt], [new_x_expr])

opti = Opti()
opti.solver('ipopt')

X = opti.variable(N+1,2)
U = opti.variable(N)
T = opti.variable()
u_m = opti.parameter()
#t_m = opti.parameter()

#cost = sum1(2+cos(X[:,0]))*T #**2
cost = T
#cost = -sum1(X[:,0])
opti.minimize(cost)

opti.subject_to(X[0,:].T == [0, 0])
opti.subject_to(cos(X[-1,0]) < -0.9999)
#opti.subject_to(T < t_m)
opti.subject_to(opti.bounded(-0.001,X[-1,1],0.001))

for ii in range(N):
    opti.subject_to(X[ii+1,:].T == new_x(X[ii,:], U[ii], T/N))
    opti.subject_to(opti.bounded(-u_m,U[ii],u_m))

In [ ]:
opti.set_initial(X[:,0], np.linspace(0, pi, N+1))
opti.set_initial(X[:,1], pi/N)
opti.set_initial(T, 50)
max_par = 0.1
opti.set_value(u_m, max_par)

sol = opti.solve()

In [ ]:
plot_results(sol.value(X), sol.value(U),sol.value(T), max_par, N)

In [ ]:
xx = integrate_euler(DM([0,0]), sol.value(U), sol.value(T)/N)
plot_results_sim(sol.value(U), xx, sol.value(T), sol.value(max_par), N)

In [ ]:
#anim = create_anim(sol)

In [ ]:
#HTML(anim.to_jshtml())

## Otras comprobaciones: Alimentar integradores avanzados con solución de Euler

In [ ]:
N = 500

k1 = F(x, u);
k2 = F(x + dt/2 * k1, u)
k3 = F(x + dt/2 * k2, u)
k4 = F(x + dt * k3, u);
new_x_expr_rk = x+dt/6*(k1 +2*k2 +2*k3 +k4)

new_x_expr_eu = x + dt * F(x, u)

new_x_expr_mid = x + dt * F(x + 0.5*dt*F(x, u), u)

new_x_eu = Function('New_x_eu', [x, u, dt], [new_x_expr_eu])
new_x_mid = Function('New_x_mid', [x, u, dt], [new_x_expr_mid])
new_x_rk = Function('New_x_rk', [x, u, dt], [new_x_expr_rk])

In [ ]:
def create_pend(new_x):
    opti = Opti()
    opti.solver('ipopt')

    X = opti.variable(N+1,2)
    U = opti.variable(N)
    T = opti.parameter()
    u_m = opti.parameter()
    #t_m = opti.parameter()

    cost = sum1(cos(X[:,0])) #**2
    #cost = T
    #cost = -sum1(X[:,0])
    opti.minimize(cost)

    opti.subject_to(X[0,:].T == [0, 0])
    

    for ii in range(N):
        opti.subject_to(X[ii+1,:].T == new_x(X[ii,:], U[ii], T/N))
        opti.subject_to(opti.bounded(-u_m,U[ii],u_m))
    return opti, X, U, T, u_m

In [ ]:
opti_eu, X, U, T, u_m = create_pend(new_x_eu)

opti_eu.subject_to(cos(X[-1,0]) < -0.9999)
#opti.subject_to(T < t_m)
opti_eu.subject_to(opti.bounded(-0.001,X[-1,1],0.001))
    
opti_eu.set_initial(X[:,0], np.linspace(0, pi, N+1))
opti_eu.set_initial(X[:,1], pi/N)
opti_eu.set_value(T, 60)
max_par = 0.1
opti_eu.set_value(u_m, max_par)

sol_eu = opti_eu.solve()

In [ ]:
X_eu = sol_eu.value(X)
U_eu = sol_eu.value(U)
T_eu = sol_eu.value(T)

In [ ]:
plot_results(X_eu, U_eu, T_eu, max_par, N)

In [ ]:
xx = integrate_rk4(DM([0,0]), U_eu, T_eu/N)
plot_results_sim(U_eu, xx, T_eu, 0.1, N)

In [ ]:
print_phase_space(X_eu)

In [ ]:
opti_mid, X, U, T, u_m = create_pend(new_x_mid)
opti_mid.set_initial(X, X_eu)
#opti_mid.set_value(T, T_eu)
opti_mid.set_value(T, 120)
opti_mid.set_initial(U, U_eu)
max_par = 0.1
opti_mid.set_value(u_m, max_par)

sol_mid = opti_mid.solve()

In [ ]:
X_mid = sol_mid.value(X)
U_mid = sol_mid.value(U)
T_mid = sol_mid.value(T)

In [ ]:
plot_results(X_mid, U_mid, T_mid, max_par, N)

In [ ]:
xx = integrate_rk4(DM([0,0]), U_mid, T_mid/N)
plot_results_sim(U_mid, xx, T_mid, 0.1, N)

In [ ]:
opti_rk, X, U, T, u_m = create_pend(new_x_rk)
opti_rk.set_initial(X, X_mid)
opti_rk.set_value(T, T_mid)
opti_rk.set_initial(U, U_mid)
max_par = 0.1
opti_rk.set_value(u_m, max_par)

sol_rk = opti_rk.solve()

In [ ]:
X_rk = sol_rk.value(X)
U_rk = sol_rk.value(U)
T_rk = sol_rk.value(T)

In [ ]:
plot_results(X_rk, U_rk, T_rk, max_par, N)

In [ ]:
xx = integrate_rk4(DM([0,0]), U_rk, T_rk/N)
plot_results_sim(U_rk, xx, T_rk, 0.1, N)

In [ ]:
print_phase_space(xx)

In [ ]:
anim = create_anim(X_rk, U_rk)

In [ ]:
HTML(anim.to_jshtml())

## Multishooting

In [ ]:
def pend_interval(new_x, X_0, U, N, dt):
    X = [X_0,]
    for ii in range(N):
        X_n = new_x(X[-1], U, dt)
        X.append(X_n.T)
    
    return vertcat(*X)

In [ ]:
N_shoot = 20
N_int = 8

options = {
    'qpsol': 'qrqp',
    'error_on_fail': False
}
opti = Opti()
opti.solver('ipopt')
#opti.solver('sqpmethod')#, options)

T_int = opti.variable(N_int)
X_transitions = opti.variable(N_int+1,2)
#U = opti.variable(N)
#dt = opti.parameter()
X_0 = opti.parameter(1,2)
u_m = opti.parameter()

cost = cos(X_transitions[-1,0]) + X_transitions[-1,1]**2#*T_int[-1] #**2
#cost = T
#cost = -sum1(X[:,0])
opti.minimize(cost)


#opti.subject_to(cos(X_transitions[-1,0]) < -0.9999)
#opti.subject_to(opti.bounded(-0.001,X_transitions[-1,1],0.001))

U = u_m
for ii in range(N_int):
    dt = T_int[ii]/N_shoot
    #X_int_arr = pend_interval(new_x_mid,X_transitions[ii,:], U, N_shoot, dt)
    X_int_arr = pend_interval(new_x_eu,X_transitions[ii,:], U, N_shoot, dt)
    U = -U
#    X_transitions[ii, :] = X_int_arr[-1, :]
    X_int_end = X_int_arr[-1, :]
    
    
    opti.subject_to(X_transitions[ii+1,:]== X_int_arr[-1, :])
    
opti.subject_to(X_transitions[0,:] == X_0)
opti.set_value(X_0, horzcat(0,0))

In [ ]:
opti.set_initial(X_transitions[:,0], np.linspace(0, pi, N_int+1))
opti.set_initial(X_transitions[:,1], 0)
opti.set_initial(T_int, np.linspace(0, 10*N_int, N_int))
max_par = 0.2
opti.set_value(u_m, max_par)

sol = opti.solve()

In [ ]:
X_arr = sol.value(X_0).reshape([1,2])
u_val = sol.value(U)
for ii in range(N_int):
    dt = sol.value(T_int)[ii]/N_shoot
    #X_int_arr = pend_interval(new_x_mid,X_transitions[ii,:], U, N_shoot, dt)
    _aa = sol.value(X_transitions)[ii,:]
    X_int_arr = pend_interval(new_x_eu, _aa.reshape([1,2]), u_val, N_shoot, dt)
    u_val = -u_val
#    X_transitions[ii, :] = X_int_arr[-1, :]
    X_arr = vertcat(X_arr, X_int_arr)

In [ ]:
X_arr

In [ ]:
plt.plot(X_arr)

In [ ]:
_ = pend_interval(new_x_mid, horzcat(0,0), 0.2, 20, 0.2)

In [ ]:
_

In [ ]:
results = []
for ii in [0.5, 0.2, 0.1, 0.05, 0.02, 0.01]:
    opti.set_value(u_m, ii)
    est_t = (2 + sqrt(1/ii))*8
    opti.set_initial(T, est_t)
    try:
        sol = opti.solve()
    except:
        pass
    else:
        results.append([sol, ii, N])

In [ ]:
for res in results:
    plot_results(*res)

In [ ]:
xx = np.array([1,0.5, 0.2, 0.1, 0.05, 0.02])
yy = np.array([1,2,3,5,6,8])

plt.plot(1/xx,yy)
#pp = np.arange(50)
pp = 1/xx
plt.plot(pp,1 + np.sqrt(pp))

In [ ]:
opti.debug.show_infeasibilities()